In [1]:
import sys
principal_path = '../'
if principal_path not in sys.path:
    sys.path.append('../')

In [2]:
import os
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torchvision.datasets import MNIST
import pytorch_lightning as pl
import torchmetrics
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from pytorch_lightning.loggers import WandbLogger
from lightning.pytorch.loggers import CSVLogger
from pytorch_lightning.loggers import TensorBoardLogger

from models.MNISTModel import MNISTModel
from models.MNISTModelWithBottelNeck import MNISTModelWithBottelNeck

In [3]:
SEED = 2024
pl.seed_everything(SEED)

Seed set to 2024


2024

In [4]:
LEARING_RATE = 1e-3
BATCH_SIZE = 64
EPOCHS = 1
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
FOLDER_CHEKPOINTS = 'checkpoints'
FOLDER_ACTIVATIONS = 'activations'
print(f'Using {DEVICE}')

Using cuda


## Loggers

In [5]:
def get_number_of_parameters(model):
    num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    unit_scale = {
        '': 1,
        'K': 10 ** 3,
        'M': 10 ** 6,
        'B': 10 ** 9
    }
    for unit, scale in sorted(unit_scale.items(), key=lambda x: x[1], reverse=True):
        if num_params >= scale:
            return f'{round(num_params / scale, 1)}{unit}'
    return str(num_params)

def log_wandb(model, model_name, folder = 'wandb_logs'):
    number_of_parameters = get_number_of_parameters(model)
    return WandbLogger(
        save_dir=folder,
        name=f"{model_name}_{number_of_parameters}",
    )

## Download Dataset

In [6]:
def load_data(batch_size=BATCH_SIZE, num_workers=4):
    # Transformaciones para los datos
    transform = transforms.ToTensor()

    # Carga de datos de entrenamiento
    mnist_train = MNIST(os.getcwd(), train=True, download=True, transform=transform)
    
    # División entre entrenamiento y validación
    train_size = int(0.8 * len(mnist_train))
    val_size = len(mnist_train) - train_size
    mnist_train, mnist_val = random_split(mnist_train, [train_size, val_size])

    # DataLoader para entrenamiento y validación
    train_loader = DataLoader(mnist_train, batch_size=batch_size, num_workers=num_workers, shuffle=True, persistent_workers=True)
    val_loader = DataLoader(mnist_val, batch_size=batch_size, num_workers=num_workers, shuffle=False, persistent_workers=True)

    # Carga de datos de test
    mnist_test = MNIST(os.getcwd(), train=False, download=True, transform=transform)
    test_loader = DataLoader(mnist_test, batch_size=batch_size, num_workers=num_workers, persistent_workers=True)

    return train_loader, val_loader, test_loader

train_loader, val_loader, test_loader = load_data()

## Training

### MNISTmodel

In [7]:
model = MNISTModel(lr=LEARING_RATE)

In [8]:
loggers = [
    log_wandb(model, 'MNISTModel'),
]

In [9]:
trainer = pl.Trainer(max_epochs=EPOCHS, logger=loggers)
trainer.fit(model, train_loader, val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: miguel_kjh (siani-ai). Use `wandb login --relogin` to force relogin
wandb: WARNING Path wandb_logs\wandb\ wasn't writable, using system temp directory.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                | Params
--------------------------------------------------
0 | train_acc | MulticlassAccuracy  | 0     
1 | val_acc   | MulticlassAccuracy  | 0     
2 | test_acc  | MulticlassAccuracy  | 0     
3 | precision | MulticlassPrecision | 0     
4 | recall    | MulticlassRecall    | 0     
5 | f1        | MulticlassF1Score   | 0     
6 | layer_1   | Linear              | 100 K 
7 | layer_2   | Linear              | 33.0 K
8 | layer_3   | Linear              | 2.6 K 
--------------------------------------------------
136 K     Trainable params
0         Non-trainable params
136 K     Total params
0.544     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]                                Failed to save model graph: 'Run' object has no attribute 'add_graph'
Epoch 0: 100%|██████████| 750/750 [00:03<00:00, 191.72it/s, v_num=kr49, train_loss=0.202, train_acc=0.922, val_loss=0.168, val_acc=0.950]Failed to save weights histogram: 'Run' object has no attribute 'add_histogram'


`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 750/750 [00:03<00:00, 191.15it/s, v_num=kr49, train_loss=0.202, train_acc=0.922, val_loss=0.168, val_acc=0.950]


In [10]:
trainer.test(model, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 157/157 [00:00<00:00, 206.24it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
           f1               0.9464820027351379
        precision           0.9515219926834106
         recall             0.9501408934593201
        test_acc            0.9519000053405762
        test_loss           0.15671443939208984
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.15671443939208984,
  'test_acc': 0.9519000053405762,
  'precision': 0.9515219926834106,
  'recall': 0.9501408934593201,
  'f1': 0.9464820027351379}]

In [11]:
## save checkpoint
file = os.path.join(FOLDER_CHEKPOINTS, 'mnist_model.ckpt')
trainer.save_checkpoint(file)

### Bottleneck

In [26]:
bottel_neck_model = MNISTModelWithBottelNeck(lr=LEARING_RATE)
print(bottel_neck_model)

MNISTModelWithBottelNeck(
  (train_acc): MulticlassAccuracy()
  (val_acc): MulticlassAccuracy()
  (test_acc): MulticlassAccuracy()
  (precision): MulticlassPrecision()
  (recall): MulticlassRecall()
  (f1): MulticlassF1Score()
  (layer_1): Linear(in_features=784, out_features=256, bias=True)
  (layer_2): Linear(in_features=256, out_features=784, bias=True)
  (layer_3): Linear(in_features=784, out_features=10, bias=True)
)


In [27]:
trainer = pl.Trainer(max_epochs=5, logger=log_wandb(bottel_neck_model, 'MNISTModelWithBottelNeck'))
trainer.fit(bottel_neck_model, train_loader, val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\code\ExAI\exai\lib\site-packages\pytorch_lightning\loggers\wandb.py:389: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
c:\code\ExAI\exai\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:639: Checkpoint directory wandb_logs\lightning_logs\qv6kkr49\checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                | Params
--------------------------------------------------
0 | train_acc | MulticlassAccuracy  | 0     
1 | val_acc   | MulticlassAccuracy  | 0     
2 | test_acc  | MulticlassAccuracy  | 0     
3 | precision | MulticlassPrecision | 0     
4 | recall    | MulticlassRecall    | 0     
5 | f1        | MulticlassF1Score   |

Training: |          | 0/? [00:00<?, ?it/s]                                 Failed to save model graph: 'Run' object has no attribute 'add_graph'
Epoch 4: 100%|██████████| 750/750 [00:03<00:00, 193.27it/s, v_num=kr49, train_loss=0.00789, train_acc=1.000, val_loss=0.115, val_acc=0.969] Failed to save weights histogram: 'Run' object has no attribute 'add_histogram'


`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 750/750 [00:03<00:00, 192.50it/s, v_num=kr49, train_loss=0.00789, train_acc=1.000, val_loss=0.115, val_acc=0.969]


In [28]:
trainer.test(bottel_neck_model, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 157/157 [00:00<00:00, 207.20it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
           f1               0.9699662923812866
        precision           0.9738726615905762
         recall             0.9718435406684875
        test_acc            0.9725000262260437
        test_loss           0.10535524040460587
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.10535524040460587,
  'test_acc': 0.9725000262260437,
  'precision': 0.9738726615905762,
  'recall': 0.9718435406684875,
  'f1': 0.9699662923812866}]

In [29]:
## save checkpoint
file = os.path.join(FOLDER_CHEKPOINTS, 'mnist_model_bottel_neck.ckpt')
trainer.save_checkpoint(file)

## Activations

In [16]:
def load_data_for_activations():
    # load only one image of the test set
    transform = transforms.ToTensor()
    mnist_test = MNIST(os.getcwd(), train=False, download=True, transform=transform)
    test_loader = DataLoader(mnist_test, batch_size=1, shuffle=False)
    return test_loader

test_loader_act = load_data_for_activations()

In [17]:
minst_model_file = os.path.join(FOLDER_CHEKPOINTS, 'mnist_model.ckpt')
model_for_act = MNISTModel.load_from_checkpoint(minst_model_file)
model_for_act.eval()

MNISTModel(
  (train_acc): MulticlassAccuracy()
  (val_acc): MulticlassAccuracy()
  (test_acc): MulticlassAccuracy()
  (precision): MulticlassPrecision()
  (recall): MulticlassRecall()
  (f1): MulticlassF1Score()
  (layer_1): Linear(in_features=784, out_features=128, bias=True)
  (layer_2): Linear(in_features=128, out_features=256, bias=True)
  (layer_3): Linear(in_features=256, out_features=10, bias=True)
)

In [18]:
def create_neuron_dataframe(layer_neurons, num_rows, test_loader_act, model_for_act):
    # Creación de las columnas para cada neurona
    columns = ['Number']
    for layer_index, neurons in enumerate(layer_neurons):
        columns += [f'layer_{layer_index+1}_Neuron{i+1}' for i in range(neurons)]

    # Pre-creación del DataFrame
    df = pd.DataFrame(index=range(num_rows), columns=columns)

    # Llenar el DataFrame
    for idx, batch in enumerate(tqdm(test_loader_act, desc='Running activations')):
        x, y = batch
        _, r = model_for_act(x, record_activations=True)

        # Construir fila para el DataFrame
        row = {'Number': y.item()}
        for layer_index, neurons in enumerate(layer_neurons):
            layer_activation = r[layer_index].cpu().detach().numpy()[0]
            for neuron_index in range(neurons):
                row[f'layer_{layer_index+1}_Neuron{neuron_index+1}'] = layer_activation[neuron_index]

        df.loc[idx] = row
    
    return df

In [19]:
# Ejemplo de uso
layer_neurons = [128, 256, 10]  # Lista con el número de neuronas en cada capa
num_rows = len(test_loader_act)  # Número de filas en el DataFrame
df = create_neuron_dataframe(layer_neurons, num_rows, test_loader_act, model_for_act)

Running activations: 100%|██████████| 10000/10000 [00:05<00:00, 1748.83it/s]


In [20]:
df.head()

,Number,layer_1_Neuron1,layer_1_Neuron2,layer_1_Neuron3,layer_1_Neuron4,layer_1_Neuron5,layer_1_Neuron6,layer_1_Neuron7,layer_1_Neuron8,layer_1_Neuron9,...,layer_3_Neuron1,layer_3_Neuron2,layer_3_Neuron3,layer_3_Neuron4,layer_3_Neuron5,layer_3_Neuron6,layer_3_Neuron7,layer_3_Neuron8,layer_3_Neuron9,layer_3_Neuron10
0,7.0,0.449542,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-12.147556,-16.15661,-7.710541,-7.271402,-16.029039,-11.763827,-21.846853,-0.001201,-14.597517,-10.055029
1,2.0,1.981847,0.0,2.332592,2.098222,0.138524,1.114918,0.0,0.0,0.37655,...,-10.453403,-7.174742,-0.005843,-5.811386,-15.197481,-7.710469,-7.108836,-16.036268,-7.167393,-18.166189
2,1.0,0.556063,0.0,0.702207,1.305601,0.0,0.185222,1.902156,0.0,0.096356,...,-10.221011,-0.01579,-6.059708,-7.541843,-6.083756,-7.373065,-6.17975,-5.144792,-6.342128,-8.537989
3,0.0,0.0,0.0,0.922271,0.0,1.780782,0.0,0.0,0.0,0.51604,...,-0.000767,-17.257816,-7.764553,-10.846607,-12.64982,-9.893709,-9.928406,-10.246429,-11.425213,-8.658001
4,4.0,0.0,0.930661,1.347643,0.0,0.0,0.0,0.0,0.907871,1.582013,...,-7.864855,-13.278628,-7.676012,-10.088933,-0.017552,-7.109054,-8.037715,-5.901061,-7.700401,-4.408139


In [22]:
# save activations csv
activation_file = os.path.join(FOLDER_ACTIVATIONS, 'activations_minist_model.csv')
df.to_csv(activation_file, index=False)

In [30]:
model_bottelnet_file = os.path.join(FOLDER_CHEKPOINTS, 'mnist_model_bottel_neck.ckpt')
model_bottelnet_for_act = MNISTModelWithBottelNeck.load_from_checkpoint(model_bottelnet_file)
model_bottelnet_for_act.eval()
print(model_bottelnet_for_act)

MNISTModelWithBottelNeck(
  (train_acc): MulticlassAccuracy()
  (val_acc): MulticlassAccuracy()
  (test_acc): MulticlassAccuracy()
  (precision): MulticlassPrecision()
  (recall): MulticlassRecall()
  (f1): MulticlassF1Score()
  (layer_1): Linear(in_features=784, out_features=256, bias=True)
  (layer_2): Linear(in_features=256, out_features=784, bias=True)
  (layer_3): Linear(in_features=784, out_features=10, bias=True)
)


In [31]:
layer_neurons = [256,784,10]
num_rows = len(test_loader_act)

df = create_neuron_dataframe(layer_neurons, num_rows, test_loader_act, model_bottelnet_for_act)
print(df.head())

Running activations: 100%|██████████| 10000/10000 [00:07<00:00, 1267.32it/s]


  Number layer_1_Neuron1 layer_1_Neuron2 layer_1_Neuron3 layer_1_Neuron4  \
0    7.0             0.0             0.0             0.0             0.0   
1    2.0             0.0        0.244701        0.372548        0.596361   
2    1.0        0.263735        0.256444             0.0         0.55511   
3    0.0             0.0             0.0        0.207519             0.0   
4    4.0             0.0             0.0             0.0             0.0   

  layer_1_Neuron5 layer_1_Neuron6 layer_1_Neuron7 layer_1_Neuron8  \
0             0.0        0.376383             0.0        0.294046   
1             0.0             0.0             0.0             0.0   
2             0.0             0.0             0.0             0.0   
3             0.0             0.0             0.0             0.0   
4        0.728545        0.652585             0.0             0.0   

  layer_1_Neuron9  ... layer_3_Neuron1 layer_3_Neuron2 layer_3_Neuron3  \
0             0.0  ...      -15.637275      -17.848444

In [25]:
# save
activation_file = os.path.join(FOLDER_ACTIVATIONS, 'activations_minist_model_bottel_neck.csv')
df.to_csv(activation_file, index=False)